In [ ]:
import pypsa
import pandas as pd
import numpy as np

In [ ]:
solver = "highs"

In [ ]:
# To calculate annualized capital expenditure
def calculate_annualised_capex(capex: float, interest: float, lifetime: int):
    crf = (
        interest * (1 + interest) ** lifetime / ((1 + interest) ** lifetime - 1)
    )  # Capital recovery factor
    return capex * crf

*** 
Check-point 1:

**Build a network in PyPSA with three nodes (`bus`) and the following components to model a simple green hydrogen production system:**

1. An `Electricity` bus with electricity as `carrier`.
   1. A wind generator connected to this bus.
   2. A solar generator connected to this bus.
2. A `Hydrogen` bus with hydrogen as `carrier`.
   1. A hydrogen demand load connected to this bus.
3. An electrolyser link to convert electricity into hydrogen.
4. A `Hydrogen_storage` bus with hydrogen as `carrier`.
   1. A hydrogen store connected to this bus.
5. A bi-directional link to transfer the hydrogen between hydrogen and hydrogen storage buses.

**For simplicity, we assume the hydrogen demand profile to be flat for now. Afterwards, we want to supply electricy by attaching one renewable power plant implemented as (`generator`) (you have to call [`network.set_snapshots`](https://pypsa.readthedocs.io/en/latest/api/_source/pypsa.Network.set_snapshots.html) to select a year). As help you should have a look at the [PyPSA documentation](https://pypsa.readthedocs.io/en/latest/) and the [minimal lopf example](https://pypsa.readthedocs.io/en/latest/examples/minimal_example_pf.html), understand what the [components documentation](https://pypsa.readthedocs.io/en/latest/user-guide/components.html) of PyPSA gives you and that you can find the underlying objective function and constraints in the [LOPF documentation](https://pypsa.readthedocs.io/en/stable/user-guide/optimal-power-flow.html).**

> **Remarks:** For time reasons, you do not have to build the network from scratch. However, to get you acquainted with PyPSA we have omitted a few elements or some of the parameters of the network marked by three question marks `???`. Either, you have to add an element similar to the one in the box above or add a few parameters.

<img src="../data/block diagrams/d2_e3-1.svg" width=600/>

Initialize network

In [ ]:
# Create empty PyPSA network
network = pypsa.Network()

In [ ]:
# Set snapshots to the year 2023 and at hourly resolution
snapshots = pd.date_range(???)
network.set_snapshots(snapshots)

In [ ]:
network.snapshots

In [ ]:
# Import an example of wind daily pattern
wind_pattern = pd.read_csv(
    "../data/weather data/example_onshore_wind_daily_pattern.csv"
)["daily pattern"]
# annual time-series availability of onshore wind (just a simplified example)
wind_profile = pd.Series(list(wind_pattern) * 365, index=network.snapshots)

In [ ]:
# Import an example of solar daily pattern
solar_pattern = pd.read_csv("../data/weather data/example_solar_daily_pattern.csv")[
    "daily pattern"
]
# annual time-series availability of solar (just a simplified example)
solar_profile = pd.Series(list(solar_pattern) * 365, index=network.snapshots)

Add an `electricity` bus with electricity as `carrier`

In [ ]:
???

Add a `hydrogen` bus with hydrogen as `carrier`

In [ ]:
???

Add a `hydrogen_storage` bus with hydrogen as `carrier`

In [ ]:
???

Add a constant hourly hydrogen load of `100MW` at the hydrogen bus. The name of the load can be `hydrogen_load`

In [ ]:
???

Add a store at the hydrogen_storage bus with a initial energy capacity of `0MWh`, and marginal cost of `0$/MWh` into the network. The name of the store can be `hydrogen_tank`. 

In [ ]:
network.add(
      class_name="Store",
      name="hydrogen_tank",
      bus=???,
      carrier=???,
      e_nom_extendable=???,
      e_cyclic=???,
      ???=0, #$/MWh
)

Add a bidirectional link, representing the hydrogen charging and discharging between hydrogen and hydrogen_storage with marginal cost of `0$/MWh` into the network? The name of the link can be `hydrogen_flow`. 

In [ ]:
network.add(
      class_name="Link",
      name="hydrogen_flow",
      bus0=???,
      bus1=???,
      p_nom_extendable=???,
      ???=0, #$/MWh
      efficiency=1,
      p_min_pu=???
)

Add a wind generator at the electricity bus with a initial capacity of `100MW`, maximum capacity of `500MW`, based on provided CAPEX, FOM, VOM, efficiency, interest rate and lifetime? The name of the generator can be `onshore_wind`. 

> **Source:** all costs for the example are taken from [PyPSA technology database](https://github.com/PyPSA/technology-data/blob/master/outputs/costs_2025.csv) and the assumptions in year 2023 in [EU map of hydrogen production costs](https://public.flourish.studio/visualisation/16659363/), with exchange rate of `1.1USD/EUR`

In [ ]:
# Onshore Wind's techno-economic parameters are given as:
eur_to_usd = 1.1
lifetime = 25
interest = 0.05

CAPEX = 1420 * eur_to_usd  # $/MW
FOM = 28 * eur_to_usd  # $/MW fixed
VOM = 0  # $/MWh variable

efficiency = 1

In [ ]:
annualized_capex = calculate_annualised_capex(CAPEX, interest, lifetime)

network.add(
    class_name=???, 
    name="onshore_wind",
    bus=???,
    carrier=???,
    p_nom_extendable=???,
    ???=100, # MW
    ???=500, # MW
    capital_cost=???, #$/MW
    marginal_cost=???, #$/MWh
    efficiency=efficiency,
    lifetime=lifetime,
    p_max_pu=wind_profile
)

Add a solar generator at the electricity bus with a initial capacity of `10MW`, maximum capacity of `500MW`, based on provided CAPEX, FOM, VOM, efficiency, interest rate and lifetime? The name of the generator can be `solar`. 

> **Source:** all costs for the example are taken from [PyPSA technology database](https://github.com/PyPSA/technology-data/blob/master/outputs/costs_2025.csv) and the assumptions in year 2023 in [EU map of hydrogen production costs](https://public.flourish.studio/visualisation/16659363/), with exchange rate of `1.1USD/EUR`

In [ ]:
# Solar's techno-economic parameters are given as:
eur_to_use = 1.1
lifetime = 20
interest = 0.05

CAPEX = 970 * eur_to_use  # $/MW
FOM = 16 * eur_to_use  # $/MW fixed
VOM = 0  # $/MWh variable

efficiency = 1

In [ ]:
annualized_capex = calculate_annualised_capex(CAPEX, interest, lifetime)

network.add(
    ???
)

Add an electrolyser link, representing the electrolysis conversion with initial capacity of `20MW`, based on provided CAPEX, FOM, VOM, efficiency, interest rate and lifetime.

> **Source:** all costs for the example are taken from [PyPSA technology database](https://github.com/PyPSA/technology-data/blob/master/outputs/costs_2025.csv) and the assumptions in year 2023 in [EU map of hydrogen production costs](https://public.flourish.studio/visualisation/16659363/), with exchange rate of `1.1USD/EUR`

In [ ]:
# Electrolyser's techno-economic parameters are given as:
eur_to_usd = 1.1
lifetime = 20
interest = 0.05

CAPEX = 1590 * eur_to_usd  # $/MW
FOM = 32 * eur_to_usd  # $/MW fixed
VOM = 0  # $/MWh variable
efficiency = 0.59

In [ ]:
annualized_capex = calculate_annualised_capex(CAPEX, interest, lifetime)

network.add(
    class_name="Link",
    name="electrolyser",
    bus0=???,
    bus1=???,
    p_nom_extendable=???,
    ???=20,
    capital_cost=???, #$/MW
    marginal_cost=???, #$/MWh
    efficiency=efficiency,
    lifetime=lifetime
)

Now try to solve your network

In [ ]:
# Solve network using cbc solver
network.optimize(pyomo=False, solver_name=solver)

Lets look at some results! Back to read the docs. How would you look at results?

In [ ]:
???

In [ ]:
# Exporting check-point 1 network
network.export_to_netcdf("../results/network_d2_e3-1.nc")

***
Check-point 2:

**Instead of the fixed hydrogen load add an hourly one at the hydrogen bus according to the daily pattern of industrial hydrogen demand (`100MW per hour` from 1am to 7pm, followed by a 5-hour shutdown). The name of the load can be `hydroge_load`**

**Then remove the electrolyser link from the previous exercise and replace it with a new electrolyser link including the compressor needed to pressurize the produced hydrogen. Cost assumptions and the efficiency of this component must be adjusted**

The block diagram representing the system remains the same as before:
<img src="../data/block diagrams/d2_e3-1.svg" width=600/>

In [ ]:
# Import check-point 1 network
network = pypsa.Network("../results/network_d2_e3-1.nc")

In [ ]:
# remove hydrogen load from previous check-point
network.remove(class_name="Load", name="hydrogen_load")

In [ ]:
# remove electrolyser link from previous check-point
network.remove(class_name="Link", name="electrolyser")

To set hourly hydrogen demand, you need to call [`network.set_snapshots`](https://pypsa.readthedocs.io/en/latest/api/_source/pypsa.Network.set_snapshots.html) to select a year. For help, please see the [PyPSA documentation](https://pypsa.readthedocs.io/en/latest/) and the [optimzation with Linopy](https://pypsa.readthedocs.io/en/latest/examples/optimization-with-linopy.html). Make sure you understand what the [components documentation](https://pypsa.readthedocs.io/en/latest/user-guide/components.html) of PyPSA gives you and that you can find the underlying objective function and constraints in the [System Optimization documentation](https://pypsa.readthedocs.io/en/stable/user-guide/optimal-power-flow.html).

In [ ]:
# Import an example of wind daily pattern
load_pattern = pd.read_csv(
    "../data/weather data/example_industrial_h2_demand_daily_pattern.csv"
)["daily pattern"]
# annual time-series availability of onshore wind (just a simplified example)
load_profile = pd.Series(list(load_pattern) * 365, index=network.snapshots)

In [ ]:
network.add(
    class_name=???,
    name=???,
    bus=???, 
    p_set=load_profile
)

Add an electrolyser link including compressor, representing the electrolysis conversion with initial capacity of `20MW`, based on provided CAPEX, FOM, VOM, water_cost, efficiency, interest rate and lifetime.

> **Source:** all costs for the example are taken from [PyPSA technology database](https://github.com/PyPSA/technology-data/blob/master/outputs/costs_2025.csv) and the assumptions in year 2023 in [EU map of hydrogen production costs](https://public.flourish.studio/visualisation/16659363/), with exchange rate of `1.1USD/EUR`

In [ ]:
# Electrolyser's techno-economic parameters are given as:
eur_to_usd = 1.1
lifetime = 20
interest = 0.05
water_cost = 1.8  # EUR/m3_H2O --> this includes the water treatment costs applied to the seawater
water_demand_ratio = (
    21  # kgH2O/kgH2 --> how much water is needed to produce 1 kg of hydrogen?
)

CAPEX = 1590 * eur_to_usd  # $/MW
FOM = 32 * eur_to_usd  # $/MW fixed
VOM = 0  # $/MWh variable
efficiency = 0.59

# Calculate marginal cost for water consumption
# marginal cost [USD/MWh] =
# water_cost [EUR/m3_H2O] * eur_to_usd [USD/EUR] / 1000 [kgH2O/m3_H2O] * water_demand_ratio [kgH2O/kgH2] / (33.33/1000 [MWh/kgH2])
water_cost = ((water_cost * eur_to_usd / 1000) * water_demand_ratio) / (
    33.33 / 1000
)  # 1 kgH2 = 33.33 kWh = 0.03333 MWh

# Add a compressor to pressurize hydrogen produced by the electrolyser and feed it to the hydrogen storage
# 1. Calculate the specific electricity consumption for compressor in kWh_el/kWh_H2. Assumption: 0.6 [kWh_el/kgH2]
# 2. Calculate the specific energy consumption for the electrolyser in kWh_el/kWh_H2. This corresponds to the reciprocal of the electrolyser's efficiency.
# 3. The sum of the two specific energy consumptions represents the total specific energy consumption for the electrolyser including compressor.
# 4. Calculate the new efficiency of the electrolyser which is the reciprocal of the total specific energy consumption.
specific_consumption_compressor = (
    0.6 / 33.33
)  # 0.6 [kWh_el/kgH2] / 33.33 [kWh_H2/kgH2] --> kWh_el/kWh_H2
specific_consumption_electrolyser = 1 / efficiency  # unit: kWh_el/kWh_H2
new_efficiency = 1 / (
    specific_consumption_compressor + specific_consumption_electrolyser
)

# Compressor's techno-economic parameters are given as:
lifetime_com = 25
CAPEX_COM = 1720 * eur_to_usd  # $/MW
FOM_COM = 34 * eur_to_usd  # $/MW fixed
VOM_COM = 0  # $/MWh variable

In [ ]:
annualized_capex = calculate_annualised_capex(CAPEX, interest, lifetime)
annualized_capex_com = calculate_annualised_capex(CAPEX_COM, interest, lifetime_com)

network.add(
    class_name=???,
    name=???,
    bus0=???,
    bus1=???,
    p_nom_extendable=???,
    ???=20,
    capital_cost=???, #$/MW
    marginal_cost=???, #$/MWh
    efficiency=new_efficiency,
    lifetime=lifetime
)

Ending of check-point 2 - Solve network, analyse results and export network

In [ ]:
# Solve network using cbc solver
network.optimize(pyomo=False, solver_name=solver)

In [ ]:
# Inspect the interaction of different power plants to supply loads
???

In [ ]:
# Inspect the change of energy capacity [MWh] in hydrogen_tank store to supply loads
???

In [ ]:
# Inspect the change of power [MW] in hydrogen_tank store to supply loads
???

In [ ]:
# Inspect the electrolysis input power
???

In [ ]:
# Inspect the charging flow
???

In [ ]:
# Exporting check-point 2 network
network.export_to_netcdf("../results/network_d2_e4-2.nc")